In [ ]:
#PROJETO FINAL 
#Algoritmo Random Forest
#Previsoes para diminuir a taxa de inadimplencia de emprestimos 

In [1]:
import psycopg2
import pandas as pd
import numpy as np
import seaborn as srn
import statistics  as sts
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
#Conexao com SQL
conn = psycopg2.connect(host="localhost",database="Emprestimo", user="postgres", password="1234", port=5432)

In [3]:
#cria um cursor
cur = conn.cursor()
CREDITO = 'public."CREDITO"'
cur.execute(f"SELECT * FROM {CREDITO}")
#executa
resultados = cur.fetchall()

In [4]:
for linha in resultados: 
    print(linha)

(2, 48, 2, 1, 5951, 2, 3, 2, 1, 3, 2, 22, 3, 1, 1, 4, 1, 0, 0, 0)
(3, 12, 1, 2, 2096, 2, 4, 2, 4, 3, 3, 49, 3, 1, 1, 3, 2, 0, 0, 1)
(4, 42, 2, 3, 7882, 2, 4, 2, 4, 4, 4, 45, 3, 2, 1, 4, 2, 0, 0, 1)
(5, 24, 3, 4, 4870, 2, 3, 3, 4, 3, 4, 53, 3, 2, 2, 4, 2, 0, 0, 0)
(6, 36, 2, 2, 9055, 1, 3, 2, 4, 3, 4, 35, 3, 2, 1, 3, 2, 1, 0, 1)
(7, 24, 2, 3, 2835, 3, 2, 3, 4, 3, 4, 53, 3, 1, 1, 4, 1, 0, 0, 1)
(9, 12, 2, 1, 3059, 4, 4, 2, 2, 3, 4, 61, 3, 1, 1, 3, 1, 0, 0, 1)
(11, 12, 2, 4, 1295, 2, 1, 3, 1, 3, 1, 25, 3, 3, 1, 4, 1, 0, 0, 0)
(13, 12, 2, 1, 1567, 2, 3, 1, 1, 3, 1, 22, 3, 1, 1, 4, 1, 1, 0, 1)
(14, 24, 1, 4, 1199, 2, 2, 4, 4, 3, 4, 60, 3, 1, 2, 3, 1, 0, 0, 0)
(16, 24, 2, 1, 1282, 5, 3, 4, 1, 3, 2, 32, 3, 1, 1, 3, 1, 0, 0, 0)
(17, 24, 1, 1, 2424, 1, 2, 4, 4, 3, 4, 53, 3, 1, 2, 4, 1, 0, 0, 1)
(18, 30, 4, 6, 8072, 1, 1, 2, 4, 3, 3, 25, 1, 1, 3, 4, 1, 0, 0, 1)
(19, 24, 2, 5, 12579, 2, 2, 4, 1, 3, 2, 44, 3, 2, 1, 1, 1, 1, 0, 0)
(20, 24, 2, 1, 3430, 3, 2, 3, 4, 3, 2, 31, 3, 1, 1, 4, 2, 1, 0, 1)
(

In [5]:
df = pd.DataFrame(resultados, columns = ["IDCREDITO","Duracao","HistoricoCredito","Proposito","Valor","Investimentos","Emprego","TempoParcelamento","EstadoCivil",
                    "FiadorTerceiros","ResidencialDesde","Idade","OutrosFinanciamentos", "Habitacao", "EmprestimoExistente", "Profissao", "Dependentes", "SocioEmpresa", "Estrangeiro", "Status"])

In [6]:
df.head()

,IDCREDITO,Duracao,HistoricoCredito,Proposito,Valor,Investimentos,Emprego,TempoParcelamento,EstadoCivil,FiadorTerceiros,ResidencialDesde,Idade,OutrosFinanciamentos,Habitacao,EmprestimoExistente,Profissao,Dependentes,SocioEmpresa,Estrangeiro,Status
0,2,48,2,1,5951,2,3.0,2,1,3,2.0,22,3,1.0,1,4,1,0,0,0
1,3,12,1,2,2096,2,4.0,2,4,3,3.0,49,3,1.0,1,3,2,0,0,1
2,4,42,2,3,7882,2,4.0,2,4,4,4.0,45,3,2.0,1,4,2,0,0,1
3,5,24,3,4,4870,2,3.0,3,4,3,4.0,53,3,2.0,2,4,2,0,0,0
4,6,36,2,2,9055,1,3.0,2,4,3,4.0,35,3,2.0,1,3,2,1,0,1


In [7]:
#Analisando dados para limpeza
agrupado = df.groupby(["HistoricoCredito"]).size()
agrupado

HistoricoCredito
0     11
1    289
2    526
3     86
4     40
5     48
dtype: int64

In [8]:
agrupado = df.groupby(["Profissao"]).size()
agrupado

Profissao
1      147
2       22
3      197
4      625
999      9
dtype: int64

In [9]:
agrupado = df.groupby(["Status"]).size()
agrupado

Status
0    300
1    700
dtype: int64

In [10]:
df["Idade"].describe()

count    1000.000000
mean       35.546000
std        11.375469
min        19.000000
25%        27.000000
50%        33.000000
75%        42.000000
max        75.000000
Name: Idade, dtype: float64

In [11]:
#Contar valores NONE
df.isnull().sum()

IDCREDITO                0
Duracao                  0
HistoricoCredito         0
Proposito                0
Valor                    0
Investimentos            0
Emprego                 10
TempoParcelamento        0
EstadoCivil              0
FiadorTerceiros          0
ResidencialDesde         7
Idade                    0
OutrosFinanciamentos     0
Habitacao                9
EmprestimoExistente      0
Profissao                0
Dependentes              0
SocioEmpresa             0
Estrangeiro              0
Status                   0
dtype: int64

In [12]:
#Substituir valor NONE "Emprego" pelo moda "3"
df["Emprego"].fillna("3.0", inplace=True)

In [13]:
df["Emprego"].isnull().sum()

0

In [14]:
moda = sts.mode(df['Emprego'])
moda

3.0

In [15]:
df.loc[df['Emprego']=='3.0', 'Emprego'] = moda
agrupado = df.groupby(['Emprego']).size()

In [16]:
agrupado

Emprego
1.0    171
2.0    247
3.0    348
4.0    172
5.0     62
dtype: int64

In [17]:
#Substituir valor NONE "ResidencialDesde" pela moda "4.0"
df["ResidencialDesde"].fillna("4.0", inplace=True)

In [18]:
df["ResidencialDesde"].isnull().sum()

0

In [19]:
moda = sts.mode(df['ResidencialDesde'])
moda

4.0

In [20]:
df.loc[df['ResidencialDesde']=='4.0', 'ResidencialDesde'] = moda
agrupado = df.groupby(['ResidencialDesde']).size()

In [21]:
agrupado

ResidencialDesde
1.0    129
2.0    308
3.0    147
4.0    416
dtype: int64

In [22]:
#Substituir valor NONE "Habitacao" pela moda 
moda = sts.mode(df['Habitacao'])
moda

1.0

In [23]:
df["Habitacao"].fillna(1.0, inplace=True)

In [24]:
agrupado = df.groupby(["Habitacao"]).size()
agrupado

Habitacao
1.0    718
2.0    107
3.0    175
dtype: int64

In [25]:
df["Habitacao"].isnull().sum()

0

In [26]:
#Colocar valores que nao existem em 'HistoricoCredito' "0" na moda 
moda = sts.mode(df['HistoricoCredito'])
moda

2

In [27]:
df.loc[df['HistoricoCredito']== 0 , 'HistoricoCredito'] = moda
agrupado = df.groupby(['HistoricoCredito']).size()

In [28]:
agrupado

HistoricoCredito
1    289
2    537
3     86
4     40
5     48
dtype: int64

In [29]:
#Colocar valores que nao existem em 'Profissao' "999" na moda 
moda = sts.mode(df['Profissao'])
moda

4

In [30]:
df.loc[df['Profissao']== 999 , 'Profissao'] = moda
agrupado = df.groupby(['Profissao']).size()

In [31]:
agrupado

Profissao
1    147
2     22
3    197
4    634
dtype: int64

In [32]:
#dados duplicados, buscamos pelo ID
df[df.duplicated(['IDCREDITO'],keep=False)]

,IDCREDITO,Duracao,HistoricoCredito,Proposito,Valor,Investimentos,Emprego,TempoParcelamento,EstadoCivil,FiadorTerceiros,ResidencialDesde,Idade,OutrosFinanciamentos,Habitacao,EmprestimoExistente,Profissao,Dependentes,SocioEmpresa,Estrangeiro,Status


In [33]:
df.head()

,IDCREDITO,Duracao,HistoricoCredito,Proposito,Valor,Investimentos,Emprego,TempoParcelamento,EstadoCivil,FiadorTerceiros,ResidencialDesde,Idade,OutrosFinanciamentos,Habitacao,EmprestimoExistente,Profissao,Dependentes,SocioEmpresa,Estrangeiro,Status
0,2,48,2,1,5951,2,3.0,2,1,3,2.0,22,3,1.0,1,4,1,0,0,0
1,3,12,1,2,2096,2,4.0,2,4,3,3.0,49,3,1.0,1,3,2,0,0,1
2,4,42,2,3,7882,2,4.0,2,4,4,4.0,45,3,2.0,1,4,2,0,0,1
3,5,24,3,4,4870,2,3.0,3,4,3,4.0,53,3,2.0,2,4,2,0,0,0
4,6,36,2,2,9055,1,3.0,2,4,3,4.0,35,3,2.0,1,3,2,1,0,1


In [35]:
#fecha conexão e cursor
cur.close()
conn.close()

In [36]:
#Fazer as previsoes
#Formato de matriz
previsores = df.iloc[:,0:19].values
classe = df.iloc[:,19].values

In [37]:
# Divisão da base de dados entre treinamento e teste (30% para testar e 70% para treinar)
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,
                                                                  classe,
                                                                  test_size = 0.30,
                                                                  random_state = 0)

In [38]:
# Criação do modelo, treinamento, obtenção das previsões e da taxa de acerto
floresta = RandomForestClassifier(n_estimators = 300)
floresta.fit(X_treinamento, y_treinamento)

RandomForestClassifier(n_estimators=300)

In [39]:
# Visualização dos atributos principais
#floresta.estimators_
floresta.estimators_[1]

DecisionTreeClassifier(max_features='sqrt', random_state=265377156)

In [40]:
previsoes = floresta.predict(X_teste)
previsoes

array([1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [41]:
# Visualização da matriz de confusão
confusao = confusion_matrix(y_teste, previsoes)
confusao

array([[ 31,  44],
       [ 19, 206]])

In [42]:
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_acerto

0.79